In [ ]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import getpass

SMTP_SERVER = "fill this"
PORT = int"fill this"
EMAIL = "fill this"
TO_EMAIL = "fill this"
PDF_FILE_PATH = "resume path"

def save_credentials():
    password = getpass.getpass("Enter your IITB email password to save (leave blank if using token): ")
    token = getpass.getpass("Enter your SSO token to save (leave blank if using password): ")
    
    with open('.env', 'w') as f:
        if password:
            f.write(f"EMAIL_PASSWORD={password}\n")
        if token:
            f.write(f"SSO_TOKEN={token}\n")
    print("Credentials saved successfully!")

def send_email_with_attachment(to_email=TO_EMAIL, subject="Research Opportunity Inquiry", name="Professor"):
    if not os.path.exists('.env'):
        save_credentials()
        
    # Load environment variables
    with open('.env', 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            os.environ[key] = value
            
    msg = MIMEMultipart()
    msg['From'] = EMAIL
    msg['To'] = to_email
    msg['Subject'] = subject
    
    with open('fill this', 'r') as f:
        body = f.read().format(name=name)
    
    msg.attach(MIMEText(body, 'plain'))
    
    with open(PDF_FILE_PATH, 'rb') as attachment:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 
                       f'attachment; filename="resume.pdf"')
        msg.attach(part)
    
    server = smtplib.SMTP(SMTP_SERVER, PORT)
    server.starttls()
    
    password = os.getenv("EMAIL_PASSWORD")
    token = os.getenv("SSO_TOKEN")
    
    try:
        if token:
            server.login("fill this", token)
        else:
            server.login("fill this", password)
        text = msg.as_string()
        server.sendmail(EMAIL, to_email, text)
        print("Email sent successfully!")
    except smtplib.SMTPAuthenticationError:
        print("Authentication failed. Please check your credentials and SSO token.")
        print("Try saving your credentials again:")
        save_credentials()
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        server.quit()
